In [1]:
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import pickle

In [2]:
data = pd.read_csv("./clean_df/clean_df.csv")

In [3]:
data = data.drop(['Unnamed: 0'],axis = 1)

In [4]:
data = data.dropna(subset=['overview','director','runtime','year']).reset_index(drop=True)

In [5]:
data = data.fillna('')

In [6]:
text_data = ['title','director','actor','overview','genres_list','key','country']
data[text_data] = data[text_data].astype(str)

In [7]:
data["key"] = data["key"].str.encode('ascii', 'ignore').str.decode('ascii')

In [8]:
def to_dummy(col,num = None):
    li = set()
    for i in range(len(data[col])):
        if num is None:
            try:
                num = len(data[col][i].split(','))
            except:
                print(data[col][i])
        for act in data[col][i].split(',')[:num]:
            li.add(act)
    li = list(li)
    for element in li:
        data[element] = data[col].astype(str).str.contains(element, case=False).astype(int)

In [9]:
dum = ['country','director','actor','genres_list','key']
for d in dum:
    if d == 'actor':
        to_dummy(d,num = 4)
    else:
        to_dummy(d)
    print(d)

country
director
actor
genres_list
key


### sklearn

In [10]:
X = data.drop(['id','title','director','actor', 'overview','genres_list','key','country'], axis=1)

In [11]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [12]:
X.shape

(4770, 19229)

In [13]:
tfidf = TfidfVectorizer(stop_words = 'english')  # initialising the TF-IDF Vector object
tfidf_matrix = tfidf.fit_transform(data['overview'])  # Constructing the TF-IDF Matrix (no. of movies x every word in vocabulary)
tfidf_matrix.shape

(4770, 20876)

In [14]:
X = hstack([X,tfidf_matrix]).toarray()

In [15]:
cosine_sim = linear_kernel(X, X)  # Constructing the Cosine Similarity Matrix (no. of movies x no. of movies)
cosine_sim.shape

(4770, 4770)

In [16]:
cosine_sim

array([[ 1.33835438e+04, -1.41439656e+00,  3.64165978e+00, ...,
        -3.80229464e+01, -3.28243197e+01, -3.07555695e+01],
       [-1.41439656e+00,  1.31553796e+04,  3.85518967e+02, ...,
        -1.51681545e+01, -2.87422235e+01, -3.75521079e+01],
       [ 3.64165978e+00,  3.85518967e+02,  1.43989299e+04, ...,
        -2.73142332e+01, -2.06045035e+01, -2.93431913e+01],
       ...,
       [-3.80229464e+01, -1.51681545e+01, -2.73142332e+01, ...,
         1.49226321e+04,  6.80475078e-01, -7.30979321e+00],
       [-3.28243197e+01, -2.87422235e+01, -2.06045035e+01, ...,
         6.80475078e-01,  8.04017231e+03,  2.05062721e+00],
       [-3.07555695e+01, -3.75521079e+01, -2.93431913e+01, ...,
        -7.30979321e+00,  2.05062721e+00,  1.21846634e+04]])

In [17]:
pickle.dump(cosine_sim, open("cosine_sim.pickle", "wb"))

In [18]:
with open("cosine_sim.pickle", 'rb') as f:
        cosine_sim = pickle.load(f)

In [19]:
indices = pd.Series(data.index, index = data['title']).drop_duplicates()

In [20]:
# Function that inputs movie titles and outputs top 10 movies similar to it

def get_recommendations(title, cosine_sim = cosine_sim):
  idx = indices[title]
  
  sim_scores = list(enumerate(cosine_sim[idx]))  # Get the similarity scores of all movies wrt input movie
  sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
  sim_scores = sim_scores[1:11]
  
  movie_indices = [i[0] for i in sim_scores]
  
  return data['title'].iloc[movie_indices]

In [21]:
get_recommendations('The Dark Knight Rises')

65                     The Dark Knight
119                      Batman Begins
2207                         12 Rounds
303                           Catwoman
95                        Interstellar
708     Maze Runner: The Scorch Trials
933                   Shanghai Knights
96                           Inception
744                     The Lego Movie
2721                 Seven Psychopaths
Name: title, dtype: object

### Use pyspark

In [22]:
spark = SparkSession.builder.getOrCreate()

22/12/16 06:35:05 WARN Utils: Your hostname, codespaces-80813b resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
22/12/16 06:35:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/16 06:35:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [23]:
data_spark=spark.createDataFrame(data)

In [24]:
from pyspark.ml.feature import Word2Vec
from pyspark.sql.functions import split
from pyspark.sql.functions import lower, col
data_spark = data_spark.withColumn("overview_splitted", split(lower(col("overview")), " "))

In [25]:
word2Vec = Word2Vec(vectorSize=100, minCount=0, maxIter=100, inputCol="overview_splitted", outputCol="features")
model = word2Vec.fit(data_spark)

22/12/16 06:50:46 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/12/16 06:50:46 WARN TaskSetManager: Stage 0 contains a task of very large size (11315 KiB). The maximum recommended task size is 1000 KiB.


22/12/16 06:50:48 ERROR Executor: Exception in task 9.0 in stage 0.0 (TID 9)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newArray(Native Method)
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2121)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1721)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2157)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1721)
	at java.base/java.io.ObjectInputStream$FieldValues.<init>(ObjectInputStream.java:2606)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2457)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2257)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1733)
	at java.base/java.io.ObjectInputStream$FieldValues.<init>(ObjectInputStream.java:2606)
	at

Py4JJavaError: An error occurred while calling o48.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 0.0 failed 1 times, most recent failure: Lost task 9.0 in stage 0.0 (TID 9) (cf9082fa-5906-43c7-86cb-03df5beff200.internal.cloudapp.net executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newArray(Native Method)
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2121)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1721)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2157)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1721)
	at java.base/java.io.ObjectInputStream$FieldValues.<init>(ObjectInputStream.java:2606)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2457)
	at java.base/java.io.ObjectInputStream.re

In [ ]:
result = model.transform(data_spark)

In [ ]:
result

In [ ]:
from pyspark.ml.feature import VectorAssembler
assemble=VectorAssembler(inputCols=[
 'popularity',
 'vote_average',
 'year'], outputCol='feature')
assembled_data=assemble.transform(data_spark)